# Handwritten character detector

In [1]:
import os
import numpy as np
import cv2
from imutils import contours

In [2]:
%matplotlib auto

Using matplotlib backend: Qt5Agg


In [5]:
path = "pictures/slika.jpg"

In [6]:
inputImage = cv2.imread(path)
inputCopy = inputImage.copy()

In [16]:
cv2.imshow("Image",inputImage)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [17]:
image = cv2.imread(path)
original = image.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (3,3), 0)
canny = cv2.Canny(blurred, 120, 255, 1)
charachters_files = []
charachters_files_bw = []

# Find contours
contour_list = []
detected_character_counter = 0
cnts = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts, _ = contours.sort_contours(cnts, method="left-to-right")

for index, contour in enumerate(cnts):
    # Obtain bounding rectangle for each contour
    # When rectangles are nested
    x,y,w,h = cv2.boundingRect(contour)
    # When current rectangle is nested in rectangle before
    if h * w > 170:
        # When current rectangle is nested in rectangle before
        if index != 0:
            x2, y2, w2, h2 = cv2.boundingRect(cnts[index-1])
            difference_x = abs(x - x2)
            if difference_x < 5 and w2 > w:
                continue 
        # When current rectangle is nested next rectangle 
        if  index != len(cnts) - 1:
            x2, y2, w2, h2 = cv2.boundingRect(cnts[index+1])
            difference_x = (x2 - x)
            if difference_x < 5 and w2 > w:
                continue 
        #print("Coordinates of bounding box are: X:" + str(x) + " Y:" + str(y) + " W:" + str(w) + " H:" + str(h))
        cropped_image = image[y:y+h, x:x+w]
        cv2.imshow("image", cropped_image)
        cv2.waitKey()
        cropped_image_copy = original[y:y+h, x:x+w]
        # Draw bounding box rectangle, crop using Numpy slicing
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),3)
        cv2.imwrite('pictures/character_{}.png'.format(detected_character_counter), cropped_image_copy)
        im_gray = cv2.imread("pictures/character_" + str(detected_character_counter) + '.png', cv2.IMREAD_GRAYSCALE)     
        (thresh, im_bw) = cv2.threshold(im_gray, 128, 255, cv2.THRESH_BINARY)
        thresh = 133
        im_bw = cv2.threshold(im_gray, thresh, 255, cv2.THRESH_BINARY)[1]
        cv2.imwrite('pictures/character_bw_'+str(detected_character_counter) + '.png', im_bw)
        charachters_files.append('pictures/character_'+str(detected_character_counter) + '.png')
        charachters_files_bw.append('pictures/character_bw_'+str(detected_character_counter) + '.png')
        detected_character_counter += 1
cv2.imshow("image", image) 
cv2.waitKey()

-1

In [16]:
for path in charachters_files:
    os.remove(path) 

In [54]:
cv2.imshow("image", image) 
cv2.waitKey()

-1

# Handwritten character classifier

In [8]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau

Using TensorFlow backend.


In [9]:
classes = ['%', '+', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '[', ']', '_']

In [18]:
# loading model
model = tf.keras.models.load_model('handwritten_character_classifier_model4.h5')

In [28]:
def get_img_array(img_path):
    img = keras.preprocessing.image.load_img(img_path, target_size=(160, 160))
    array = keras.preprocessing.image.img_to_array(img)  * (1.0 / 255)
    array = tf.image.rgb_to_grayscale(array)
    slika = keras.preprocessing.image.array_to_img(array)
    array = np.expand_dims(array, axis=0)
    return array

In [20]:
expression = ""

In [27]:
expression = ""
for i, character_file in enumerate(charachters_files_bw):
    inputImage = cv2.imread(character_file)
    img_array = get_img_array(character_file)
    index_of_class = np.argmax(model.predict(img_array)[0])
    character = classes[index_of_class]
    expression = expression[0:len(expression)] + character
print("Detected expression: " + expression)

Detected expression: 3-5+6-8


# Solver

In [31]:
def calculate(first_number, second_number, operator):
    if operator == "+":
        return float(first_number) + float(second_number)
    elif operator == "-":
        return float(first_number) - float(second_number)
    elif operator == "*":
        return float(first_number) * float(second_number)
    else:
        return float(first_number) / float(second_number)
    
def check_for_presence_of_operators(expression):
#function return 1 if math operators are present in expression, or 0 if math operators are not present
    if expression.find("+") == -1 and expression.find("-") == -1 and expression.find("*") == -1 and expression.find("/") == -1:
        return 0
    return 1

def check_for_presence_of_brackets(expression):
#function return 1 if bracket is present in expression, or 0 if bracket is not present
    if expression.find("(") == -1:
        return 0
    return 1

def check_for_multiplication_or_divison(expression):
#function return 0 if multiplication or divison are not present in expression, or 0 if they are not present
    if expression.find("*") == -1 and expression.find("/") == -1:
        return 0
    return 1

def check_for_addition_or_subtraction(expression):
#function return 0 if addition or subtraction are not present in expression, or 0 if they are not present
    if expression.find("+") == -1 and expression.find("-") == -1:
        return 0
    return 1

def get_first_index_of_multiplication_or_division(expression):
#function return index of multiplication or division dependetly to what appear first
    presence_of_multiplication = expression.find("*")
    presence_of_division = expression.find("/")
    if presence_of_multiplication == -1 and presence_of_division == -1:
        return 0
    if presence_of_multiplication == -1:
        return expression.index("/")
    elif presence_of_division == -1:
        return expression.index("*")
    else:
        if expression.index("*") < expression.index("/"):
            return expression.index("*")
        else:
            return expression.index("/")

def get_first_index_of_addition_or_subtraction(expression):
#function return index of multiplication or division dependetly to what appear first
    presence_of_addition = expression.find("+")
    presence_of_subtraction = expression.find("-")
    if presence_of_addition == -1 and presence_of_subtraction == -1:
        return -1
    if presence_of_addition == -1:
        return expression.index("-")
    elif presence_of_subtraction == -1:
        return expression.index("+")
    else:
        if expression.index("+") < expression.index("-"):
            return expression.index("+")
        else:
            return expression.index("-")
        
def get_last_index_of_addition_or_subtraction(expression):
#function return index of multiplication or division dependetly to what appear first
    presence_of_addition = expression.rfind("+")
    presence_of_subtraction = expression.rfind("-")
    if presence_of_addition == -1 and presence_of_subtraction == -1:
        return -1
    if presence_of_addition == -1:
        return expression.rfind("-")
    elif presence_of_subtraction == -1:
        return expression.rfind("+")
    else:
        if expression.rfind("+") > expression.rfind("-"):
            return expression.rfind("+")
        else:
            return expression.rfind("-")
        
def get_second_index_of_addition_or_subtraction(expression):
    index_of_first_occurence_of_addition_or_subtraction = get_first_index_of_addition_or_subtraction(expression)
    substring = expression[index_of_first_occurence_of_addition_or_subtraction+1:len(expression)]
    index_of_second_occurence_of_addition_or_subtraction = index_of_first_occurence_of_addition_or_subtraction + get_first_index_of_addition_or_subtraction(substring) + 1
    return index_of_second_occurence_of_addition_or_subtraction

def get_index_of_operator_right_to_given_operator(expression, index):
#function return first index of operators
    substring = expression[index+1:len(expression)]
    indexes_of_operators = [substring.find("*"), substring.find("/"), substring.find("+"), substring.find("-")]
    #get rid of -1 values from indexes_of_operators array
    indexes_of_operators = list(filter(lambda index : index != -1, indexes_of_operators))
    return -1 if len(indexes_of_operators) == 0 else min(indexes_of_operators)

def check_if_there_is_one_operator(expression):
#function return 1 if there is one operator in expression
    if (expression.count("+") + expression.count("-") + expression.count("*") + expression.count("/")) == 1:
        return 1
    else:
        return 0

def get_index_of_operator_left_to_given_operator(expression, index):
    substring = expression[0:index]
    addition_index = substring.rfind("+")
    subtraction_index = substring.rfind("-")
    if max(addition_index, subtraction_index) == -1:
        return -1
    else:
        return max(addition_index, subtraction_index)

def get_index_of_opening_bracket_left_to_given_index(expression, index):
    substring = expression[0:index]
    return substring.rfind("(")

def get_index_of_any_operator_left_to_given_operator(expression, index):
    substring = expression[1:index]
    addition_index = substring.rfind("+")
    subtraction_index = substring.rfind("-")
    multiplication_index = substring.rfind("*")
    division_index = substring.rfind("/")
    return max(addition_index, subtraction_index, multiplication_index, division_index)

def get_left_part_of_substring(expression, first_index_of_multiplication_or_division):
# in case of using division or multiplication we have concatenate part of string left to the first operator
# 1) when there is no addition or subtraction (also can be -a) a * (or /) b  * (or /) c ... (or /) 
# 2) expression could be in format - a + b ... - c * (or /) d
# 3) expression could be in format - a + b ... + c * (or /) c
# 4) expression could be in format - a * (/) b * (or /) c ...
# 5) expression could be in format - a * (/) b + (or -) c ...
# 6) expression could be in format a + (or -) b + c * (or /) d ..
# 7) expression could be in format a + (or -) b - c * (or /) d ..
# 8) expression could be in format a * (/) b + (-) c
#    expression in format -a *(/) b is already implemented
    # if there is no addition or subtraction value that is returned is 0
    presence_of_addition = 0 if get_first_index_of_addition_or_subtraction(expression[1:len(expression)]) == -1 else 1
    substring = expression[first_index_of_multiplication_or_division+1:len(expression)]
    # when expression is in form 1)
    if presence_of_addition == 0:
        return ""
    if (expression[0] == "-" and (get_first_index_of_addition_or_subtraction(expression[1:len(expression)]) < first_index_of_multiplication_or_division)):
        index_of_operator_left_to_multiplication_or_division = get_index_of_operator_left_to_given_operator(expression, first_index_of_multiplication_or_division)
        # when expression is in form 2)
        if (expression[index_of_operator_left_to_multiplication_or_division] == "-"):
            #print("#2")
            return expression[0:index_of_operator_left_to_multiplication_or_division]
        # when expression is in form 3)
        else:
            #print("#3")
            return expression[0:index_of_operator_left_to_multiplication_or_division + 1]
    # when expression is in form 4) or when expression is in form 5)
    if (expression[0] == "-" and (get_first_index_of_addition_or_subtraction(expression[1:len(expression)]) > first_index_of_multiplication_or_division)):
        #print("#4 and #5")
        return ""
    # when expression is in form 6
    if get_first_index_of_addition_or_subtraction(expression) < get_first_index_of_multiplication_or_division(expression):
        index_of_operator_left_to_multiplication_or_division = get_index_of_operator_left_to_given_operator(expression, first_index_of_multiplication_or_division)
        if expression[index_of_operator_left_to_multiplication_or_division] == "-":
            #print("#6")
            return expression[0:index_of_operator_left_to_multiplication_or_division]
        else:
            #print("#7")
            return expression[0:index_of_operator_left_to_multiplication_or_division+1]
    # when expression is in form 8
    if (first_index_of_multiplication_or_division < get_first_index_of_addition_or_subtraction(expression[1:len(expression)])):
        #print("#8")
        return ""

def find_nth_occurence_of_substring(expression, substring, n):
    current_index = expression.find(substring)
    while current_index >= 0 and n > 1:
        current_index = expression.find(substring, current_index+len(substring))
        n -= 1
    return current_index

def get_right_part_of_substring(expression, first_index_of_multiplication_or_division):
# in case of using division or multiplication we have concatenate part of string right to the first operator
    index_of_operator_right_to_given_operator = get_index_of_operator_right_to_given_operator(expression, first_index_of_multiplication_or_division)
    if index_of_operator_right_to_given_operator == -1:
        return ""
    else:
        return expression[1+first_index_of_multiplication_or_division+get_index_of_operator_right_to_given_operator(expression, first_index_of_multiplication_or_division):len(expression)]
    
def number_right_to_given_operator(expression, first_index_of_multiplication_or_division):
# in case of using division or multiplication we have to get right number
    # 1) when there is no addition or subtraction (also can be -a) a * (or /) b  * (or /) c ... * (or /) 
    # 2) expression could be in format - (or +) a + (or -) b ... - c * (or /) d
    #    substring = expression[1:index] because first character could be "-"
    index_of_operator_right_to_given_operator = get_index_of_operator_right_to_given_operator(expression, first_index_of_multiplication_or_division)
    #indexes_of_operators = [expression.find("*"), expression.find("/"), expression.find("+"), expression.find("-")]
    #indexes_of_operators_with_minus_one_value = list(filter(lambda index : index == -1, indexes_of_operators))
    #indexes_of_operators_without_minus_one_value = list(filter(lambda index : index != -1, indexes_of_operators))
    if index_of_operator_right_to_given_operator == -1:
        return float(expression[first_index_of_multiplication_or_division+1:len(expression)])
    else:
        return float(expression[first_index_of_multiplication_or_division+1:first_index_of_multiplication_or_division+index_of_operator_right_to_given_operator+1])
    
def number_left_to_given_operator(expression, first_index_of_multiplication_or_division):
# in case of using division or multiplication we have to get left number
    # 1) when there is no addition or subtraction (also can be -a) a * (or /) b  * (or /) c ... * (or /) 
    # 2) expression could be in format - (or +) a + (or -) b ... - c * (or /) d
    #    substring = expression[1:index] because first character could be "-"
    substring = expression[1:first_index_of_multiplication_or_division]
    addition_index = substring.rfind("+")
    subtraction_index = substring.rfind("-")
    if max(addition_index, subtraction_index) == -1:
        first_number = float(expression[0:first_index_of_multiplication_or_division])
    else:
        if expression[max(addition_index, subtraction_index)] == "-":
            return float(expression[max(addition_index, subtraction_index):first_index_of_multiplication_or_division])
        else:
            return float(expression[max(addition_index, subtraction_index)+1:first_index_of_multiplication_or_division])
            
    return first_number

def evaluate(expression):
    if check_if_there_is_one_operator(expression[1:len(expression)]) == 1:
        if check_for_multiplication_or_divison(expression[1:len(expression)]) == 1:
            first_index_of_multiplication_or_division = get_first_index_of_multiplication_or_division(expression)
            operator = expression[first_index_of_multiplication_or_division]
            first_number = float(expression[0:first_index_of_multiplication_or_division])
            second_number = float(expression[first_index_of_multiplication_or_division+1:len(expression)])
            expression = str(calculate(first_number, second_number, operator))
        else:
            if expression[0] == "-":
                first_index_of_addition_or_subtraction = get_first_index_of_addition_or_subtraction(expression[1:len(expression)]) + 1
            else:
                first_index_of_addition_or_subtraction = get_first_index_of_addition_or_subtraction(expression)
            operator = expression[first_index_of_addition_or_subtraction]
            first_number = float(expression[0:first_index_of_addition_or_subtraction])
            second_number = float(expression[first_index_of_addition_or_subtraction+1:len(expression)])
            expression = str(calculate(first_number, second_number, operator))
        return expression
    else:
        if check_for_multiplication_or_divison(expression) == 1:
            first_index_of_multiplication_or_division = get_first_index_of_multiplication_or_division(expression)
            first_number = number_left_to_given_operator(expression, first_index_of_multiplication_or_division)
            second_number = number_right_to_given_operator(expression, first_index_of_multiplication_or_division)
            operator = expression[first_index_of_multiplication_or_division]
            result = str(calculate(first_number, second_number, operator))
            left_part_of_substring = get_left_part_of_substring(expression, first_index_of_multiplication_or_division)
            right_part_of_substring = get_right_part_of_substring(expression, first_index_of_multiplication_or_division)
            expression = left_part_of_substring + result + right_part_of_substring
            return expression
        else:
            if expression[0] == "-":
                first_index_of_addition_or_subtraction = get_first_index_of_addition_or_subtraction(expression[1:len(expression)]) + 1
                second_index_of_addition_or_subtraction = get_second_index_of_addition_or_subtraction(expression[1:len(expression)]) + 1
            else:
                first_index_of_addition_or_subtraction = get_first_index_of_addition_or_subtraction(expression)
                second_index_of_addition_or_subtraction = get_second_index_of_addition_or_subtraction(expression)
            first_number = float(expression[0:first_index_of_addition_or_subtraction])
            second_number = float(expression[first_index_of_addition_or_subtraction+1:second_index_of_addition_or_subtraction])
            operator = expression[first_index_of_addition_or_subtraction]
            result = str(calculate(first_number, second_number, operator))            
            substring_after_second_number = expression[second_index_of_addition_or_subtraction:len(expression)]
            expression = str(result + substring_after_second_number)
            return expression
        
def evaluate_expression_with_parenthesis(expression):
    # here last found means last in context of getting it, not last last as in context as last character in string
    index_of_last_found_opening_bracket = expression.find("(")
    index_of_last_found_closing_bracket = expression.find(")")
    if (expression[index_of_last_found_opening_bracket+1:len(expression)].find("(") != -1):
        index_of_last_found_second_opening_bracket = 1 + index_of_last_found_opening_bracket + expression[index_of_last_found_opening_bracket+1:len(expression)].find("(")
    else:
        index_of_last_found_second_opening_bracket = -1        
    # case wheen there are no nested brackets
    if index_of_last_found_second_opening_bracket == -1 or (index_of_last_found_closing_bracket < index_of_last_found_second_opening_bracket):
        result = evaluate(expression[index_of_last_found_opening_bracket+1:index_of_last_found_closing_bracket])
        if check_for_presence_of_operators(result[1:len(result)]) == 1:
            left_part_of_substring = expression[0:index_of_last_found_opening_bracket+1]
            right_part_of_substring = expression[index_of_last_found_closing_bracket:len(expression)]
            expression = left_part_of_substring + result + right_part_of_substring
        else:
            right_part_of_substring = expression[index_of_last_found_closing_bracket+1:len(expression)]
            if expression[index_of_last_found_opening_bracket-1] == "-" and result[0] == "-":
                result = result[1:len(result)]
                left_part_of_substring = expression[0:index_of_last_found_opening_bracket-1]+"+"
                expression = left_part_of_substring + result + right_part_of_substring
                #right_part_of_substring[len(right_part_of_substring)] = "+"
                #right_part_of_substring = [0:len(right_part_of_substring)-1] + "+"
            elif expression[index_of_last_found_opening_bracket-1] == "-" and result[0] != "-":
                left_part_of_substring = expression[0:index_of_last_found_opening_bracket-1]+"-"
                expression = left_part_of_substring + result + right_part_of_substring
            elif expression[index_of_last_found_opening_bracket-1] == "+" and result[0] != "-":
                left_part_of_substring = expression[0:index_of_last_found_opening_bracket]
                expression = left_part_of_substring + result + right_part_of_substring
            elif expression[index_of_last_found_opening_bracket-1] == "+" and result[0] == "-":
                left_part_of_substring = expression[0:index_of_last_found_opening_bracket-1]
                expression = left_part_of_substring + result + right_part_of_substring
            elif (expression[index_of_last_found_opening_bracket-1] == "*" or expression[index_of_last_found_opening_bracket-1] == "/") and result[0] != "-":
                left_part_of_substring = expression[0:index_of_last_found_opening_bracket]
                expression = left_part_of_substring + result + right_part_of_substring
            elif (expression[index_of_last_found_opening_bracket-1] == "*" or expression[index_of_last_found_opening_bracket-1] == "/") and result[0] == "-":
                #index_of_character_that_multiply_or_divide_number_in_bracket = get_index_of_any_operator_left_to_given_operator(expression, index_of_last_found_opening_bracket-1)+1
                index_of_character_preciding_multiplication_or_division = get_index_of_operator_left_to_given_operator(expression, index_of_last_found_opening_bracket-1)
                sign_of_character_preciding_multiplication_or_divisiont = expression[index_of_character_preciding_multiplication_or_division]
                if (index_of_character_preciding_multiplication_or_division == -1):
                    left_part_of_substring = "-" + expression[0:index_of_last_found_opening_bracket]
                    result = result[1:len(result)]
                    #print("ulazim 1")
                    expression = left_part_of_substring + result + right_part_of_substring
                elif sign_of_character_preciding_multiplication_or_divisiont == "+":
                    #print("ulazim 2")
                    left_part_of_substring = expression[0:index_of_character_preciding_multiplication_or_division] + "-" + expression[index_of_character_preciding_multiplication_or_division+1:index_of_last_found_opening_bracket]
                    #print("******************")
                    #print("******************")
                    result = result[1:len(result)]
                    expression = left_part_of_substring + result + right_part_of_substring
                elif index_of_character_preciding_multiplication_or_division == 0 and sign_of_character_preciding_multiplication_or_divisiont == "-":
                    #print("***")
                    #print("ulazim 3")
                    index_of_character_preciding_multiplication_or_division = get_index_of_operator_left_to_given_operator(expression, index_of_last_found_opening_bracket-1)
                    left_part_of_substring = expression[1:index_of_last_found_opening_bracket]
                    result = result[1:len(result)]
                    expression = left_part_of_substring + result + right_part_of_substring
                elif index_of_character_preciding_multiplication_or_division != 0 and sign_of_character_preciding_multiplication_or_divisiont == "-":
                    #print("ulazim 4")
                    left_part_of_substring = expression[0:index_of_character_preciding_multiplication_or_division] + "+" + expression[index_of_character_preciding_multiplication_or_division+1:index_of_last_found_opening_bracket]
                    result = result[1:len(result)]
                    expression = left_part_of_substring + result + right_part_of_substring
    else:
        # index_of_last_found_closing_bracket is index of first found ")"
        substring = expression[0:index_of_last_found_closing_bracket]
        # index of opening bracket positioned left to index of last found closing bracket 
        index_of_opening_bracket = substring.rfind("(")
        expression_within_brackets = expression[index_of_opening_bracket + 1:index_of_last_found_closing_bracket]
        result = evaluate(expression_within_brackets)
        if check_for_presence_of_operators(result[1:len(result)]) == 1:
            left_part_of_substring = expression[0:index_of_opening_bracket+1]
            right_part_of_substring = expression[index_of_last_found_closing_bracket:len(expression)]
            expression = left_part_of_substring + result + right_part_of_substring
            print(expression)
        else:
            right_part_of_substring = expression[index_of_last_found_closing_bracket+1:len(expression)]
            if expression[index_of_opening_bracket-1] == "(":
                left_part_of_substring = expression[0:index_of_opening_bracket]
                expression = left_part_of_substring + result + right_part_of_substring 
            elif expression[index_of_opening_bracket-1] == "-" and result[0] == "-":
                result = result[1:len(result)]
                left_part_of_substring = expression[0:index_of_opening_bracket-1]+"+"
                expression = left_part_of_substring + result + right_part_of_substring
                print(expression)
            elif expression[index_of_opening_bracket-1] == "-" and result[0] != "-":
                left_part_of_substring = expression[0:index_of_opening_bracket-1]+"-"
                expression = left_part_of_substring + result + right_part_of_substring
            elif expression[index_of_opening_bracket-1] == "+" and result[0] != "-":
                left_part_of_substring = expression[0:index_of_opening_bracket]
                expression = left_part_of_substring + result + right_part_of_substring
            elif expression[index_of_opening_bracket-1] == "+" and result[0] == "-":
                left_part_of_substring = expression[0:index_of_opening_bracket-1]
                expression = left_part_of_substring + result + right_part_of_substring
            elif (expression[index_of_opening_bracket-1] == "*" or expression[index_of_opening_bracket-1] == "/") and result[0] != "-":
                left_part_of_substring = expression[0:index_of_opening_bracket]
                expression = left_part_of_substring + result + right_part_of_substring
            elif (expression[index_of_opening_bracket-1] == "*" or expression[index_of_opening_bracket-1] == "/") and result[0] == "-":
                index_of_opening_bracket_left_to_given_index = get_index_of_opening_bracket_left_to_given_index(expression, index_of_opening_bracket)
                last_index_of_addition_or_subtraction_in_parenthesis_expression = get_last_index_of_addition_or_subtraction(expression[index_of_opening_bracket_left_to_given_index+1:index_of_opening_bracket])
                index_of_addition_or_subtraction = index_of_opening_bracket_left_to_given_index + last_index_of_addition_or_subtraction_in_parenthesis_expression
                sign_of_character_preciding_multiplication_or_divisiont = expression[index_of_addition_or_subtraction + 1]
                if last_index_of_addition_or_subtraction_in_parenthesis_expression == -1:
                    left_part_of_substring = expression[0:index_of_opening_bracket_left_to_given_index+1]+"-" + expression[index_of_opening_bracket_left_to_given_index+1:index_of_opening_bracket]
                    expression = left_part_of_substring + result[1:len(result)] + right_part_of_substring
                elif sign_of_character_preciding_multiplication_or_divisiont == "-" and last_index_of_addition_or_subtraction_in_parenthesis_expression != 0:
                    left_part_of_substring = expression[0:index_of_opening_bracket_left_to_given_index + last_index_of_addition_or_subtraction_in_parenthesis_expression + 1] + "+" + expression[index_of_opening_bracket_left_to_given_index + last_index_of_addition_or_subtraction_in_parenthesis_expression + 2:index_of_opening_bracket]
                    expression = left_part_of_substring + result[1:len(result)] + right_part_of_substring
                elif sign_of_character_preciding_multiplication_or_divisiont == "-" and last_index_of_addition_or_subtraction_in_parenthesis_expression == 0:
                    left_part_of_substring = expression[0:index_of_opening_bracket_left_to_given_index + last_index_of_addition_or_subtraction_in_parenthesis_expression + 1] + expression[index_of_opening_bracket_left_to_given_index + last_index_of_addition_or_subtraction_in_parenthesis_expression + 2:index_of_opening_bracket]
                    expression = left_part_of_substring + result[1:len(result)] + right_part_of_substring
                elif sign_of_character_preciding_multiplication_or_divisiont == "+":
                    left_part_of_substring = expression[0:index_of_opening_bracket_left_to_given_index + last_index_of_addition_or_subtraction_in_parenthesis_expression + 1] + "-" + expression[index_of_opening_bracket_left_to_given_index + last_index_of_addition_or_subtraction_in_parenthesis_expression + 2:index_of_opening_bracket]
                    expression = left_part_of_substring + result[1:len(result)] + right_part_of_substring
    return expression

In [32]:
input_expression = expression
while check_for_presence_of_operators(expression[1:len(expression)]) == 1:
    found_most_nested_brackets = 0
    while check_for_presence_of_brackets(expression) == 1:
        expression = evaluate_expression_with_parenthesis(expression)
    expression = evaluate(expression)

In [34]:
print(input_expression + "=" + expression)

3-5+6-8=-4.0
